<a href="https://colab.research.google.com/github/maciekpiotrowski/dw_matrix_car/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [2]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [8]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [4]:
df=pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT='_cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):continue
  factorized_values=df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat]=factorized_values
  else:
    df[feat+SUFFIX_CAT]=factorized_values

In [0]:
cat_feats=[x for x in df.columns if SUFFIX_CAT in x]
cat_feats=[x for x in cat_feats if 'price' not in x]

In [0]:
def run_model(model=DecisionTreeRegressor(max_depth=5),feats=cat_feats):
  x=df[feats].values
  y=df['price_value'].values
  scores=cross_val_score(model,x,y,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [10]:
run_model()

(-19695.13091100928, 148.72570644015937)

In [0]:
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x:-1 if str(x)=='None' else int(x))
df['param_moc']=df['param_moc'].map(lambda x:-1 if str(x)=='None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x:-1 if str(x)=='None' else int(x.split('cm')[0].replace(' ','')))

In [16]:
feats = ['param_napęd_cat','param_rok-produkcji','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa','seller_name_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat','feature_czujniki-parkowania-przednie_cat','feature_łopatki-zmiany-biegów_cat','feature_regulowane-zawieszenie_cat']
xgparams={
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}
model=xgb.XGBRegressor(**xgparams)
run_model(model, feats)

[16:44:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:44:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:44:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)